In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import detector.detect as detect

%matplotlib inline

from IPython.display import Image, Video
from pathlib import Path

In [2]:
#read video
source = 'ring-video.mp4'
cap = cv2.VideoCapture(source)

if not cap.isOpened():
    print("Error opening video stream or file")

In [3]:
#store frames in array
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    
    if not ret:
        break

    frame = cv2.resize(frame, None, fx = 0.3, fy = 0.3)
    frames.append(frame)

cap.release()

In [4]:
#create widgets & displays
frameslider = widgets.IntSlider(
    value = 84,
    min = 0,
    max = len(frames) - 1,
    description = 'frame:')

hue = widgets.IntRangeSlider(
    value = [7, 35],
    min = 0,
    max = 255,
    description = 'Hue :')

sat = widgets.IntRangeSlider(
    value = [65, 255],
    min = 0,
    max = 255,
    description = 'Sat :')

val = widgets.IntRangeSlider(
    value = [205, 255],
    min = 0,
    max = 255,
    description = 'Val :')



def display_frames(frame):
    image = frames[frame]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)

def display_processed(frame, hue, sat, val):
    image = frames[frame].copy()
    matches = detect.detect(image, low = np.array([hue[0], sat[0], val[0]]), high = np.array([hue[1], sat[1], val[1]]))
    for m in matches:
        image = m.show(image)
        #image = Match.show(m, image)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
        

output = widgets.interactive_output(display_frames, {'frame': frameslider})
processed_output = widgets.interactive_output(display_processed, {'frame': frameslider, 'hue': hue, 'sat': sat, 'val': val})

hsv_values = widgets.HBox([hue, sat, val])
displays = widgets.HBox([output, processed_output]) 
sliders = widgets.HBox([frameslider, hsv_values])
widgets.VBox([sliders, displays])


In [5]:
frame_width = frames[0].shape[1]
frame_height = frames[0].shape[0]

out_mp4 = cv2.VideoWriter("detection.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 24, (frame_width, frame_height))

for frame in frames:
    matches = detect.detect(frame, low = np.array([7, 65, 200]), high = np.array([35, 255, 255]))
    frame = frame.copy()
    
    for m in matches:
        frame = m.show(frame)
        #image = Match.show(m, image)

    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    out_mp4.write(frame)
    
out_mp4.release()
print(frame_width, frame_height)

!ffmpeg -y -i "detection.mp4" -c:v libx264 "detection_x264.mp4"  -hide_banner -loglevel error

576 324


In [6]:
Video(filename = "detection_x264.mp4")